In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census & gmaps API Keys
#from config import (api_key, gkey)
#c = Census(api_key, year=2010)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [9]:
# Create unemployment dataframe for San Francisco, CA from USDA csv
unemploy_CA_df = pd.read_csv('Unemployment_SF.csv', header=None)
unemploy_CA_df.dropna(inplace = True) 
unemploy_CA_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
1,FIPS,Name,2011,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,Median Household Income (2019),% of State Median HH Income
2,06000,California,11.7,10.4,8.9,7.5,6.2,5.5,4.8,4.3,4.0,"$80,423",100.0%
3,06001,"Alameda County, CA",10.1,8.7,7.2,5.8,4.7,4.3,3.7,3.1,2.9,"$107,589",133.8%
4,06003,"Alpine County, CA",12.8,11.2,9.7,8.2,7.3,6.3,6.0,4.6,5.1,"$58,112",72.3%
5,06005,"Amador County, CA",13.6,12.3,10.3,8.3,6.6,5.9,5.0,4.1,3.8,"$62,640",77.9%


In [ ]:
# Delete extraneous columns
#del df.loc['FIELD8']
#df.head()

In [11]:
# Create unemployment dataframe for Baton Rouge, LA from USDA csv
unemploy_LA_df = pd.read_csv('Unemployment_BR.csv', header=None)
unemploy_LA_df.dropna(inplace = True) 
unemploy_LA_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
1,FIPS,Name,2011,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,Median Household Income (2019),% of State Median HH Income
2,22000,Louisiana,7.8,7.1,6.7,6.4,6.3,6.1,5.1,4.9,4.8,"$51,108",100.0%
3,22001,"Acadia Parish, LA",6.7,6.0,6.0,5.8,6.9,7.4,5.9,5.4,5.3,"$44,728",87.5%
4,22003,"Allen Parish, LA",9.1,8.0,7.5,7.2,7.0,6.4,5.3,5.7,5.7,"$47,111",92.2%
5,22005,"Ascension Parish, LA",7.0,6.1,5.6,5.3,5.1,4.9,4.3,4.1,4.1,"$83,072",162.5%


In [ ]:
#FROM THIS CELL AND BELOW - COPIED FROM CLASS ACTIVITY AND STILL NEED TO EDIT

In [3]:
#example of reading tab separated file - see census
#pandas.read_csv(filename, sep='\t', lineterminator='\r')

In [4]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
#census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
#                           "B19301_001E",
#                           "B17001_002E",
#                           "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
#census_pd = pd.DataFrame(census_data)

# Column Reordering
#census_pd = census_pd.rename(columns={"B01003_001E": "Population",
#                                       "B01002_001E": "Median Age",
#                                       "B19013_001E": "Household Income",
#                                       "B19301_001E": "Per Capita Income",
#                                       "B17001_002E": "Poverty Count",
#                                       "B23025_005E": "Unemployment Count",
#                                       "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
#census_pd["Poverty Rate"] = 100 * \
#    census_pd["Poverty Count"].astype(
#        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
# census_pd["Unemployment Rate"] = 100 * \
#     census_pd["Unemployment Count"].astype(
#         int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

NameError: name 'census_pd' is not defined

In [ ]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

In [ ]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("../Resources/state_centroids.csv")
centroids.head()

In [ ]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")

# Save the updated dataframe as a csv
census_data.to_csv("../Resources/state_census_data.csv", encoding="utf-8", index=False)
census_data.head()

In [ ]:
# Convert poverty rate as a list
# Convert bank rate to list
poverty_rate = census_data["Poverty Rate"].tolist()

In [ ]:
# Create a map using state centroid coordinates to set markers
marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

![10-State_Markers.png](../Images/10-State_Markers.png)